# YouTube Audio Crawler - Google Colab (Cookie Auth Version)

This notebook downloads audio from YouTube channels with cookie authentication to bypass bot detection.

**Requirements:**
- Format: M4A (AAC codec)
- Sample Rate: 44000 Hz (exact)
- Channels: Mono (1 channel)
- Bitrate: 192kbps

## ⚠️ IMPORTANT: Cookie Authentication Required

Without cookies, YouTube will show: **"Sign in to confirm you're not a bot"**

### How to get YouTube cookies:

**Method 1: Chrome Extension (Easiest)**
1. Install: [Get cookies.txt LOCALLY](https://chrome.google.com/webstore/detail/get-cookiestxt-locally/cclelndahbckbenkjhflpdbgdldlbecc)
2. Go to https://www.youtube.com and login
3. Click extension icon → Export
4. Save as `youtube_cookies.txt`
5. Upload to Colab in Step 2

**Method 2: Firefox Extension**
1. Install: [cookies.txt](https://addons.mozilla.org/en-US/firefox/addon/cookies-txt/)
2. Go to https://www.youtube.com and login
3. Export cookies
4. Save as `youtube_cookies.txt`
5. Upload to Colab in Step 2

## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -q yt-dlp ffmpeg-python

# FFmpeg is pre-installed in Colab
!ffmpeg -version | head -n 1

print("\n✅ Dependencies installed successfully")

## Step 2: Upload YouTube Cookies (REQUIRED)

**Upload your `youtube_cookies.txt` file here:**

In [ ]:
from google.colab import files
import os

print("📤 Please upload your youtube_cookies.txt file:")
print("   (Export from browser using cookie extension)\n")

uploaded = files.upload()

# Check if cookies file was uploaded
COOKIES_FILE = None
for filename in uploaded.keys():
    if 'cookie' in filename.lower() or filename.endswith('.txt'):
        COOKIES_FILE = filename
        print(f"\n✅ Cookies file uploaded: {filename}")
        
        # Verify file content
        with open(filename, 'r') as f:
            lines = f.readlines()
            if any('youtube.com' in line for line in lines):
                print(f"✅ Valid YouTube cookies detected ({len(lines)} lines)")
            else:
                print("⚠️  Warning: No youtube.com cookies found in file")
        break

if not COOKIES_FILE:
    print("\n❌ No cookies file uploaded!")
    print("   Downloads will likely fail with 'Sign in to confirm you're not a bot'")
    print("   Please upload youtube_cookies.txt file")

## Step 3: Setup and Configuration

In [ ]:
import yt_dlp
import json
import time
import random
from pathlib import Path
from datetime import datetime
from urllib.parse import urlparse

# Create downloads directory
DOWNLOADS_DIR = Path('./downloads')
DOWNLOADS_DIR.mkdir(exist_ok=True)

# Rate limiting configuration (conservative settings)
SLEEP_BETWEEN_VIDEOS = 15  # seconds between each video
SLEEP_MIN = 10  # minimum random sleep
SLEEP_MAX = 20  # maximum random sleep
RATE_LIMIT = '200K'  # 200KB/s download speed

print("✅ Setup complete")
print(f"📁 Downloads directory: {DOWNLOADS_DIR.absolute()}")
print(f"🍪 Cookies file: {COOKIES_FILE if COOKIES_FILE else 'None (⚠️  downloads may fail)'}")

## Step 4: Core Download Functions (With Cookie Support)

In [ ]:
def get_channel_videos(channel_url, use_cookies=True):
    """Get all video URLs from a channel"""
    print(f"\n🔍 Fetching videos from: {channel_url}")
    
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,
        'force_generic_extractor': False,
    }
    
    # Add cookies if available
    if use_cookies and COOKIES_FILE:
        ydl_opts['cookiefile'] = COOKIES_FILE
        print(f"   🍪 Using cookies: {COOKIES_FILE}")
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(channel_url, download=False)
            
            if 'entries' in result:
                videos = []
                for entry in result['entries']:
                    if entry:
                        video_id = entry.get('id')
                        video_url = f"https://www.youtube.com/watch?v={video_id}"
                        videos.append({
                            'video_id': video_id,
                            'url': video_url,
                            'title': entry.get('title', 'Unknown')
                        })
                
                print(f"   ✅ Found {len(videos)} videos")
                return videos
            
    except Exception as e:
        error_msg = str(e)
        print(f"   ❌ Error fetching channel videos: {error_msg}")
        
        if 'bot' in error_msg.lower():
            print(f"   ⚠️  Bot detection! Make sure cookies are uploaded and valid.")
        
        return []
    
    return []

def download_video_audio(video_info, use_cookies=True):
    """Download audio from a single video with cookie authentication"""
    video_id = video_info['video_id']
    video_url = video_info['url']
    title = video_info['title']
    
    print(f"\n⬇️  Downloading: {title[:50]}...")
    print(f"   Video ID: {video_id}")
    
    # Create video directory
    video_dir = DOWNLOADS_DIR / video_id
    video_dir.mkdir(exist_ok=True)
    
    output_path = video_dir / f"{video_id}.%(ext)s"
    
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': str(output_path),
        'writeinfojson': False,
        'no_warnings': True,
        'ignoreerrors': False,
        'ratelimit': RATE_LIMIT,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'm4a',
            'preferredquality': '192',
        }],
        'postprocessor_args': [
            '-ar', '44000',  # 44kHz sample rate (exact)
            '-ac', '1',      # Mono channel
        ],
    }
    
    # Add cookies if available
    if use_cookies and COOKIES_FILE:
        ydl_opts['cookiefile'] = COOKIES_FILE
        print(f"   🍪 Using authenticated session")
    
    try:
        # Random sleep before download
        sleep_time = random.uniform(SLEEP_MIN, SLEEP_MAX)
        print(f"   ⏳ Sleeping {sleep_time:.1f}s before download...")
        time.sleep(sleep_time)
        
        # Download
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=True)
            
            # Save metadata
            metadata = {
                'video_id': video_id,
                'title': info.get('title'),
                'description': info.get('description'),
                'uploader': info.get('uploader'),
                'upload_date': info.get('upload_date'),
                'duration': info.get('duration'),
                'view_count': info.get('view_count'),
                'like_count': info.get('like_count'),
                'channel_id': info.get('channel_id'),
                'channel_url': info.get('channel_url'),
                'url': video_url,
                'audio_format': 'm4a',
                'sample_rate': 44000,
                'channels': 1,
                'bitrate': 192,
                'downloaded_at': datetime.now().isoformat(),
                'authentication': 'cookies' if (use_cookies and COOKIES_FILE) else 'none'
            }
            
            metadata_path = video_dir / f"{video_id}.json"
            with open(metadata_path, 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, ensure_ascii=False)
            
            print(f"   ✅ Downloaded successfully")
            
            # Sleep between videos
            print(f"   ⏳ Sleeping {SLEEP_BETWEEN_VIDEOS}s before next video...")
            time.sleep(SLEEP_BETWEEN_VIDEOS)
            
            return True, None
            
    except Exception as e:
        error_msg = str(e)
        print(f"   ❌ Error: {error_msg}")
        
        # Check error types
        if '403' in error_msg or 'Forbidden' in error_msg:
            print(f"   ⚠️  403 Forbidden - YouTube blocked this request")
        elif 'bot' in error_msg.lower():
            print(f"   ⚠️  Bot detection - Check cookies are valid and not expired")
        
        return False, error_msg

print("✅ Functions loaded with cookie support")

## Step 5: Configuration - Add Your Channel URLs

In [ ]:
# Configure your channels here
CHANNELS = [
    "https://www.youtube.com/channel/UCLFgJS-f6UKOJ3Xz0K8Kosg",  # leon
    "https://www.youtube.com/channel/UC9nijyKbu2cQ0lrK6RyGLsw",  # 詩詩fly
    # "https://www.youtube.com/channel/UC74T0OeGBT2bOcidVMwqqoQ",  # Ross_Liu
]

# Max videos per channel (for testing, set to None for all videos)
MAX_VIDEOS_PER_CHANNEL = 5  # Download only first 5 videos per channel for testing

print(f"📋 Configured {len(CHANNELS)} channels")
print(f"📊 Max videos per channel: {MAX_VIDEOS_PER_CHANNEL if MAX_VIDEOS_PER_CHANNEL else 'ALL'}")

## Step 6: Start Download Process

In [ ]:
# Statistics
stats = {
    'total_videos': 0,
    'successful': 0,
    'failed': 0,
    'forbidden_403': 0,
    'bot_detection': 0,
    'start_time': datetime.now()
}

print("🚀 Starting download process...")
print(f"⏰ Start time: {stats['start_time'].strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🍪 Using cookies: {'Yes' if COOKIES_FILE else 'No (⚠️  may fail)'}")
print("=" * 60)

for channel_url in CHANNELS:
    print(f"\n{'='*60}")
    print(f"📺 Processing channel: {channel_url}")
    print(f"{'='*60}")
    
    # Get videos from channel (with cookies)
    videos = get_channel_videos(channel_url, use_cookies=True)
    
    if not videos:
        print("⚠️  No videos found or error fetching channel")
        continue
    
    # Limit videos if configured
    if MAX_VIDEOS_PER_CHANNEL:
        videos = videos[:MAX_VIDEOS_PER_CHANNEL]
        print(f"📊 Limited to first {len(videos)} videos")
    
    # Download each video
    for i, video_info in enumerate(videos, 1):
        print(f"\n--- Video {i}/{len(videos)} ---")
        stats['total_videos'] += 1
        
        success, error = download_video_audio(video_info, use_cookies=True)
        
        if success:
            stats['successful'] += 1
        else:
            stats['failed'] += 1
            if error:
                if '403' in error or 'Forbidden' in error:
                    stats['forbidden_403'] += 1
                elif 'bot' in error.lower():
                    stats['bot_detection'] += 1

# Final statistics
stats['end_time'] = datetime.now()
stats['duration'] = (stats['end_time'] - stats['start_time']).total_seconds()

print("\n" + "="*60)
print("📊 FINAL STATISTICS")
print("="*60)
print(f"⏰ Start time: {stats['start_time'].strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏰ End time: {stats['end_time'].strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️  Duration: {stats['duration']:.0f} seconds ({stats['duration']/60:.1f} minutes)")
print(f"\n📹 Total videos attempted: {stats['total_videos']}")
print(f"✅ Successful downloads: {stats['successful']}")
print(f"❌ Failed downloads: {stats['failed']}")
print(f"🚫 403 Forbidden errors: {stats['forbidden_403']}")
print(f"🤖 Bot detection errors: {stats['bot_detection']}")

if stats['total_videos'] > 0:
    success_rate = (stats['successful'] / stats['total_videos']) * 100
    print(f"\n📈 Success rate: {success_rate:.1f}%")
    
    if success_rate == 0:
        print("\n⚠️  TROUBLESHOOTING:")
        if stats['bot_detection'] > 0:
            print("   - Bot detection errors: Check cookies are valid and not expired")
            print("   - Try exporting fresh cookies from YouTube")
        if stats['forbidden_403'] > 0:
            print("   - 403 errors: May need IP rotator service (paid solution)")
        if not COOKIES_FILE:
            print("   - No cookies uploaded! Upload youtube_cookies.txt in Step 2")

print("\n✅ Process complete!")

## Step 7: View Results

In [ ]:
# List all downloaded files
print("📂 Downloaded files:\n")

successful_downloads = 0
failed_downloads = 0
total_size_mb = 0

for video_dir in sorted(DOWNLOADS_DIR.iterdir()):
    if not video_dir.is_dir():
        continue
    
    m4a_files = list(video_dir.glob('*.m4a'))
    json_files = list(video_dir.glob('*.json'))
    
    if m4a_files:
        successful_downloads += 1
        m4a_file = m4a_files[0]
        size_mb = m4a_file.stat().st_size / (1024 * 1024)
        total_size_mb += size_mb
        print(f"✅ {video_dir.name}/")
        print(f"   └─ {m4a_file.name} ({size_mb:.2f} MB)")
        if json_files:
            print(f"   └─ {json_files[0].name}")
    else:
        failed_downloads += 1
        print(f"❌ {video_dir.name}/ (incomplete - no .m4a file)")

print(f"\n📊 Summary:")
print(f"   ✅ Successful: {successful_downloads}")
print(f"   ❌ Failed: {failed_downloads}")
print(f"   💾 Total size: {total_size_mb:.2f} MB")
if successful_downloads > 0:
    print(f"   📏 Average size: {total_size_mb/successful_downloads:.2f} MB per file")

## Step 8: Download Files (Optional)

Create a ZIP file of all successfully downloaded audio files:

In [ ]:
# Create ZIP file of all downloads
import shutil

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
zip_filename = f'youtube_audio_downloads_{timestamp}'

print(f"📦 Creating ZIP file: {zip_filename}.zip")
shutil.make_archive(zip_filename, 'zip', DOWNLOADS_DIR)
print(f"✅ ZIP created successfully!")
print(f"\n📥 Download the file from the files panel on the left")
print(f"   File: {zip_filename}.zip")

# Also offer direct download
try:
    files.download(f"{zip_filename}.zip")
    print(f"\n⬇️  Download started automatically!")
except:
    print(f"\n💡 If auto-download failed, right-click the file in left panel → Download")